In [ ]:
from utils import app_settings, MinioClient
from temporalio.client import Client
from pathlib import Path
from temporal_workflow.workflows import ExtractionPDF
import uuid

pdf_paths = [Path("./attention_is_all_you_need.pdf")] * 1000

# 1. Initialiser le client MinIO
minio_client = MinioClient()

# 2. Connexion au serveur Temporal
print(f"🔌 Connexion à Temporal sur {app_settings.TEMPORAL_HOST}...")
temporal_client = await Client.connect(app_settings.TEMPORAL_HOST)

# 3. Upload des PDFs et lancement des workflows
print(f"\n🚀 Upload de {len(pdf_paths)} PDFs et lancement des workflows...\n")

workflow_handles = []

for i, pdf_path in enumerate(pdf_paths, 1):
    # Upload du PDF dans MinIO avec un nom unique
    pdf_name = f"pdf_{i}_{pdf_path.name}"
    object_name = f"input_pdfs/{pdf_name}"

    print(f"[{i}/{len(pdf_paths)}] Upload de {pdf_path.name}...")
    minio_client.upload_file(pdf_path, object_name)
    print(f"    ✅ Uploadé: {object_name}")

    # Lancer le workflow
    workflow_id = f"pdf-extraction-{i}-{uuid.uuid4()}"

    print(f"    🚀 Lancement du workflow {workflow_id}...")
    handle = await temporal_client.start_workflow(
        ExtractionPDF.run,
        object_name,
        id=workflow_id,
        task_queue="pdf-extraction",
    )

    workflow_handles.append((pdf_name, handle))
    print(
        f"    ✅ Workflow lancé: http://localhost:8080/namespaces/default/workflows/{handle.id}\n"
    )

# 4. Récapitulatif
print("\n" + "=" * 80)
print(f"✅ {len(workflow_handles)} workflows lancés avec succès!")
print("=" * 80)
print("\n📊 Liste des workflows:")
for pdf_name, handle in workflow_handles:
    print(f"  • {pdf_name}")
    print(f"    → http://localhost:8080/namespaces/default/workflows/{handle.id}\n")

print("\n💡 Conseils:")
print("  • Ouvrez http://localhost:8080 pour suivre les workflows dans Temporal UI")
print("  • Assurez-vous que les workers (worker-pdf et worker-ml) sont démarrés")
print("  • Utilisez 'docker-compose logs -f worker-pdf worker-ml' pour voir les logs")

🔌 Connexion à Temporal sur localhost:7233...

🚀 Upload de 1000 PDFs et lancement des workflows...

[1/1000] Upload de attention_is_all_you_need.pdf...
    ✅ Uploadé: input_pdfs/pdf_1_attention_is_all_you_need.pdf
    🚀 Lancement du workflow pdf-extraction-1-6609954d-cf19-423c-9668-28bb9aea6631...
    ✅ Workflow lancé: http://localhost:8080/namespaces/default/workflows/pdf-extraction-1-6609954d-cf19-423c-9668-28bb9aea6631

[2/1000] Upload de attention_is_all_you_need.pdf...
    ✅ Uploadé: input_pdfs/pdf_2_attention_is_all_you_need.pdf
    🚀 Lancement du workflow pdf-extraction-2-6ac1a92e-1458-42b2-afde-83d4576ee642...
    ✅ Workflow lancé: http://localhost:8080/namespaces/default/workflows/pdf-extraction-2-6ac1a92e-1458-42b2-afde-83d4576ee642

[3/1000] Upload de attention_is_all_you_need.pdf...
    ✅ Uploadé: input_pdfs/pdf_3_attention_is_all_you_need.pdf
    🚀 Lancement du workflow pdf-extraction-3-4d1b2188-6b44-48cb-9f5c-fb15d09b33e1...
    ✅ Workflow lancé: http://localhost:8080/name